In [2]:
import pandas as pd
import altair as alt

Defaulting to user installation because normal site-packages is not writeable
  Using cached altair-5.0.0rc1-py3-none-any.whl (709 kB)
     |████████████████████████████████| 55 kB 4.3 MB/s eta 0:00:01
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
df = pd.read_csv("tmdb_5000_movies.csv")
df.info()

NameError: name 'pd' is not defined

In [8]:
df = pd.read_csv("tmdb_5000_movies.csv")

df["budget_in_millions"] = df["budget"] / 1000000
df["revenue_in_millions"] = df["revenue"] / 1000000


df= df [["budget_in_millions", "revenue_in_millions", "title", "release_date", "vote_count"]]
df.to_csv("scatter.csv")

In [13]:
df = pd.read_csv("tmdb_5000_movies.csv")

df["revenue_in_millions"] = df["revenue"] / 1000000

def genre_name(row):
    index = row["genres"].find("\"name\": \"") + 9
    string = row["genres"][index:]
    index_end = string.find("\"")
    string = string[:index_end]
    return string


df["genres_name"] = df.apply(lambda row: genre_name(row), axis=1)

def year_of_release(row):
    date_str = str(row["release_date"])
    index = date_str.find("-")
    return date_str[:index]

df["year_of_release"] = df.apply(lambda row: year_of_release(row), axis=1)

df = df[df.genres_name != ""]
df = df[df.year_of_release != ""]

year_filter = df[(df["year_of_release"] >= "1980") & (df["year_of_release"] <= "2015")]
gener_list = ["Drama", "Comedy", "Action", "Adventure", "Horror", "Romance", "Crime", "Thriller", "Animation", "Fantasy"]
genre_filter = year_filter[year_filter.genres_name.isin(gener_list)]


genre_filter = genre_filter[["genres_name", "year_of_release", "revenue_in_millions"]]
genre_filter.to_csv("line.csv")

In [6]:
df = pd.read_csv("scatter.csv")

interval = alt.selection_interval(encodings=['x'])

scatter = alt.Chart(df).mark_point().encode(
    alt.X("budget_in_millions:Q", axis=alt.Axis(title='Budget in Millions')),
    alt.Y("revenue_in_millions:Q", axis=alt.Axis(title='Revenue in Millions')),
    tooltip=["title", "release_date", "vote_count"],
    color = alt.condition(alt.expr.datum['budget_in_millions'] < alt.expr.datum['revenue_in_millions'], 
                          alt.value('darkgreen'), alt.value('slategray'))
).properties(
    width=700,
    height = 500,
    title={"text": "Budget vs Revenue of the Top 5000 Rated Movies",
          "subtitle": "Green means the film made money, grey means it lost money. Use the tick chart to filter films by number of votes"}
).transform_filter(
    interval
)

tick = alt.Chart(df).mark_tick().encode(
    alt.X("vote_count", axis=alt.Axis(title='Number of Votes')),
    color = alt.value('gray'),
    #opacity = alt.value(0.4)
).properties(
    width = 700
)

view = tick.add_params(
    interval
)

combo = scatter & view
combo.save('scatter_2.html')

In [8]:
def genre_name(row):
    index = row["genres"].find("\"name\": \"") + 9
    string = row["genres"][index:]
    index_end = string.find("\"")
    string = string[:index_end]
    return string


df["genres_name"] = df.apply(lambda row: genre_name(row), axis=1)

def year_of_release(row):
    date_str = str(row["release_date"])
    index = date_str.find("-")
    return date_str[:index]

df["year_of_release"] = df.apply(lambda row: year_of_release(row), axis=1)

df = df[df.genres_name != ""]
df = df[df.year_of_release != ""]

In [15]:
genre_filter = pd.read_csv("line.csv")

selection = alt.selection_point(fields=['genres_name'], bind='legend')

revenue = alt.Chart(genre_filter).mark_line().encode(
    x = alt.X("year_of_release", axis = alt.Axis(title='Year of Release')),
    y = alt.Y("sum(revenue_in_millions)", axis = alt.Axis(title='Revenue in Millions')),
    color = "genres_name"
).properties(
    width=600,
    height=300,
    title = {"text": "Film revenue and number of films released over time",
          "subtitle": "Click the legend to filter revenue by genre"}
).transform_filter(
    selection
)

stream = alt.Chart(genre_filter).mark_area().encode(
    x = alt.X("year_of_release", axis = alt.Axis(title='Year of Release')),
    y="count()",
    color = "genres_name",
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
).properties(
    width=600,
    height=300
).add_params(
    selection
)

line = (revenue & stream)
line.save('line_2.html')